In [1]:
pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 14.7 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=738338 sha256=b565fe2ba589e46c5e37db1a106eba7359fb3c27267b1c8f7cd67f50d44d02ed
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm


In [2]:
from lightfm import LightFM
from scipy.sparse import coo_matrix as sp
import time
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import csv
import requests
from itertools import islice
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split 

import numpy as np
import pandas as pd
import warnings # warings 제거
warnings.filterwarnings(action='ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# import json

# with open('/content/drive/My Drive/steam/steam_store_data.json', 'r') as f:
#     json_data = json.load(f)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 1. Data structure and embedding




> ## Data loading, preprocessing




**user_features** : user의 특성을 구분(성별, 지역, 선호유형)

**item_features**: item의 meta data(제목, 가격, 장르, 국가, 게시 시간)

**interactions** : user-item간의 data(평점)

In [7]:
#author -  user_id
#appid - game
#voted_up - true means it was a positive recommendation
#votes_up - the number of users that found this review helpful
## votes_up integer로 해야 함

# data_overten = pd.read_csv('/content/drive/My Drive/steam/parsed_data_overten1.csv')
data_overten = {'author' : ['1','2', '3', '4', '5', '6', '7'],
                'appid' : ['10', '10', '20', '20', '30', '30', '30'],
                'playtime_forever' : ['104', '426', '386', '165', '28', '45', '30'], 
                'playtime_at_review' : ['10', '426', '199', '184', '40', '21', '31'],
                'voted_up' : ['True', 'Ture', 'Ture', 'Ture', 'False', 'False', 'False'],
                'votes_up' : [0, 0, 3, 4, 1, 2, 1]}

game_data = {'name' : ['A', 'B', 'C'],
             'steam_appid' : ['10', '20', '30']}
# users : user_features
users = pd.DataFrame(data_overten, columns = ['author'])
# ratings : intersections
# ratings = pd.DataFrame(data_overten, columns = ['author', 'appid', 'playtime_forever', 'playtime_at_review'])
ratings = pd.DataFrame(data_overten)
# games : game_features
games = pd.DataFrame(game_data, columns = ['steam_appid', 'name'])



> ## Data embedding
위의 데이터를 embedding dataset으로 build



In [11]:
## make array 
# interaction
ratings_source = [(ratings['author'][i], 
                   ratings['appid'][i],
                   ratings['votes_up'][i]) for i in range(ratings.shape[0])]
# item_feautres
item_meta = games[['steam_appid', 'name']]
item_features_source = [(item_meta['steam_appid'][i],
                        np.array(item_meta.iloc[i,1:]).reshape(-1))
                        for i in range(item_meta.shape[0])]
# user_features
user_meta = users[['author']]
user_features_source = [(user_meta['author'][i],
                        np.array(user_meta.iloc[i,1:]).reshape(-1)) 
                        for i in range(user_meta.shape[0])]   

## fit dataset 
dataset = Dataset()
dataset.fit(users=ratings['author'],
            items=ratings['appid'])
interactions, weights = dataset.build_interactions(ratings_source)

dataset.fit(users=ratings['author'],
            items=ratings['appid'],
            item_features=item_meta[item_meta.columns[1:]].values.flatten())
item_features=dataset.build_item_features(item_features_source)

dataset.fit(users=ratings['author'],
            items=ratings['appid'],
            user_features=user_meta[user_meta.columns[1:]].values.flatten())
user_features=dataset.build_user_features(user_features_source)

# embedding data 는 print(item_features) 
print(item_features)

  (0, 0)	0.5
  (0, 3)	0.5
  (1, 1)	0.5
  (1, 4)	0.5
  (2, 2)	0.5
  (2, 5)	0.5


# 2. LightFM Model Function



> ## Model fit, optimize



loss function 으로  WARP ,  정확도는 precision@k 를 사용하여 evalutation 을 해준다.

In [12]:
from lightfm.cross_validation import random_train_test_split
# Split Train, Test data
model = LightFM(loss='warp')
(train, test) = random_train_test_split(interactions=interactions, test_percentage=0.2)
model.fit(train, item_features=item_features,user_features=user_features, epochs=2)

train_precision = precision_at_k(model, train, k=5, item_features=item_features,user_features=user_features).mean()
test_precision = precision_at_k(model, test, k=5, item_features=item_features,user_features=user_features).mean()
train_auc = auc_score(model, train, item_features=item_features,user_features=user_features).mean()
test_auc = auc_score(model, test, item_features=item_features,user_features=user_features).mean()
# test_auc = auc_score(model, test, item_features=item_features).mean()
output = -test_precision #precision은 높을수록 좋음 : 높은 숫자를 찾기위해 음수화

print( 'Precision: train %.4f, test %.4f, AUC: train %.4f, test %.4f.' 
      % ( train_precision,test_precision,train_auc, test_auc))

Precision: train 0.2000, test 0.2000, AUC: train 0.5000, test 0.2500.


추천에서는 성능 지표로 precision@k 를 많이 사용하고 

Precision@5 = 0.05 (test) 는 순위로 rank 가 된 게임 중 (k=) 5번째 내에 관심 있을 게임이 나올 정확도가 평균 5% 라는 것입니다. 수치가 매우 낮아보이지만, user/content 수가 많은 real 환경에서 5%를 넘기는 어렵다.
​

AUC는 모델의 전체 적인 성능을 나타내는 지표이다.

​



> ## Predict score & Recommend games



In [15]:
labels = np.array(games['steam_appid'])

def sample_recommendation(model, data, user_ids):

    n_users, n_items = data.shape

    #build a structure to store user scores for each item
    all_scores = np.empty(shape=(0,n_items))

    #iterate through the group and build the scores
    for user_id in user_ids:
        #known_positives = labels[data.tocsr()[user_id].indices]
        #user의 전체 0~9066 movie의 예측 평점
        scores = model.predict(user_id,np.arange(n_items),item_features) 
        
        # movieId로 치환
        top_items_for_user = labels[np.argsort(-scores)] #argsort():오름차순인덱스반환
        print("Top Recommended games For User: ", user_id)
        for x in top_items_for_user[:10]:
            #print("     %s" % games[games['gameId']== x])
            print("     %s" % x)

        #vertically stack the user scores (items are columns)
        all_scores = np.vstack((all_scores, scores))
        print(all_scores)

    #compute the average rating for each item in the group
    item_averages = np.mean(all_scores.astype(np.float), axis=0)
    top_items_for_group = labels[np.argsort(-item_averages)]
    top_items_scores=item_averages[np.argsort(-item_averages)]

    print("Top Recommended games for Group:")

    for x in top_items_for_group[:10]:
        print("     %s" % x)
    return  top_items_for_group, top_items_scores



*   group에 author id를 입력하면 모든 games 의 예측 평점을 계산하고
ex) user 1 -> group = [0], user 2 = group = [1] 
*   list_no에 원하는 게임 수를 입력하면 높은순으로 games(id:score)를 리턴합니다.

In [21]:
# predict for existing user
# 
# list_no : 
group = [5]
list_no = 2

#sample recommendations for the group
list,avg_scores = sample_recommendation(model, interactions, group)
# pd.DataFrame(list).to_csv('movie.csv')
recommend_movie=pd.DataFrame(columns=games.columns)
for x in list[:list_no]:
  recommend_movie=recommend_movie.append(games[games['steam_appid']== x], ignore_index=True)

recommend_movie['predict_score']= avg_scores[:list_no]
recommend_movie

Top Recommended games For User:  5
     20
     10
     30
[[-0.03579989 -0.01650534 -0.05171992]]
Top Recommended games for Group:
     20
     10
     30


,steam_appid,name,predict_score
0,20,B,-0.016505
1,10,A,-0.035800
